In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/corated-dataset/solar_lentigo/ISIC_3261677.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_1855712.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_9433685.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_1110208.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_5473320.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_9660231.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_2990668.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_4105969.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_9105966.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_4366243.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_3637517.jpg
/kaggle/input/corated-dataset/solar_lentigo/ISIC_0889556.jpg
/kaggle/input/corated-dataset/basal_cell_carcinoma/ISIC_0028059.jpg
/kaggle/input/corated-dataset/basal_cell_carcinoma/ISIC_0031986.jpg
/kaggle/input/corated-dataset/basal_cell_carcinoma/ISIC_0029352.jpg
/kaggle/input/corated-dataset/basal_cell_carcinoma/ISIC_0033366.

In [2]:
import os

# Silence TF/XLA/CUDA warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"   # warnings+
os.environ["XLA_FLAGS"] = "--xla_gpu_strict_conv_algorithm_picker=false"


In [3]:
import os
import numpy as np
import tensorflow as tf
from collections import Counter



2026-01-03 14:47:04.339003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767451624.851992      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767451624.978609      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767451626.142678      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767451626.142721      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767451626.142724      24 computation_placer.cc:177] computation placer alr

In [4]:
DATASET_DIR = "/kaggle/input/corated-dataset"

IMG_SIZE = (224, 224)
BATCH_SIZE = 16        # Safe for Kaggle GPU
EPOCHS_STAGE_1 = 15
EPOCHS_STAGE_2 = 10
SEED = 42



In [5]:
import shutil
import os

SRC_DATASET = "/kaggle/input/corated-dataset"
DST_DATASET = "/kaggle/working/corated-dataset"

# copy only once
if not os.path.exists(DST_DATASET):
    shutil.copytree(SRC_DATASET, DST_DATASET)
    print("Dataset copied to working directory")
else:
    print("Dataset already exists in working directory")



Dataset copied to working directory


In [6]:
merge_map = {
    "verruca_warts": "warts",
    "squamous_cell_carcinoma": "basal_cell_carcinoma",
    "solar_lentigo": "dermatofibroma"
}


In [7]:
for source_class, target_class in merge_map.items():
    source_path = os.path.join(DST_DATASET, source_class)
    target_path = os.path.join(DST_DATASET, target_class)

    if not os.path.exists(source_path):
        print(f"Source not found: {source_class}")
        continue

    os.makedirs(target_path, exist_ok=True)

    for img in os.listdir(source_path):
        src = os.path.join(source_path, img)
        dst = os.path.join(target_path, f"{source_class}_{img}")
        shutil.move(src, dst)

    print(f"Merged {source_class} → {target_class}")


Merged verruca_warts → warts
Merged squamous_cell_carcinoma → basal_cell_carcinoma
Merged solar_lentigo → dermatofibroma


In [8]:
for source_class in merge_map.keys():
    source_path = os.path.join(DST_DATASET, source_class)
    if os.path.exists(source_path) and len(os.listdir(source_path)) == 0:
        os.rmdir(source_path)
        print(f"Deleted empty folder: {source_class}")


Deleted empty folder: verruca_warts
Deleted empty folder: squamous_cell_carcinoma
Deleted empty folder: solar_lentigo


In [9]:
DATASET_PATH = "/kaggle/working/corated-dataset"


In [10]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)

print("Classes:", class_names)
print("Number of classes:", NUM_CLASSES)


Found 20523 files belonging to 15 classes.


I0000 00:00:1767451876.050404      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1767451876.054383      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Classes: ['acne_rosacea', 'actinic_keratosis', 'bacterial_infections', 'basal_cell_carcinoma', 'dermatofibroma', 'eczema', 'melanoma', 'nevus', 'psoriasis', 'scabies_bites', 'seborrheic_keratosis', 'tinea_fungal', 'urticaria', 'vascular_lesions', 'warts']
Number of classes: 15


In [11]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.25),
    tf.keras.layers.RandomZoom(0.25),
    tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.RandomBrightness(0.2),
])


In [12]:
counter = Counter()

for i, cls in enumerate(class_names):
    cls_path = os.path.join(DATASET_DIR, cls)
    counter[i] = len([
        f for f in os.listdir(cls_path)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

total = sum(counter.values())
class_weight = {
    i: total / (NUM_CLASSES * counter[i])
    for i in counter
}

print("Class distribution:", counter)
print("Class weights:", class_weight)


Class distribution: Counter({7: 7148, 5: 2152, 8: 1757, 11: 1625, 14: 1358, 6: 1257, 10: 1170, 0: 882, 13: 748, 3: 677, 9: 539, 1: 366, 2: 361, 12: 265, 4: 126})
Class weights: {0: 1.5442932728647014, 1: 3.721493624772313, 2: 3.7730378578024006, 3: 2.011915312653865, 4: 10.81005291005291, 5: 0.6329306071871128, 6: 1.0835852559002916, 7: 0.19055213579556052, 8: 0.7752229178523999, 9: 2.527025355596784, 10: 1.1641595441595443, 11: 0.8381948717948718, 12: 5.139874213836478, 13: 1.820944741532977, 14: 1.0029945999018164}


In [13]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, models

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # 🔒 Phase 1


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,386,482 (16.73 MB)

 Trainable params: 334,351 (1.28 MB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [16]:
# 1) Model builds
model.summary()

# 2) One training step works
model.fit(train_ds.take(1))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,386,482 (16.73 MB)

 Trainable params: 334,351 (1.28 MB)

 Non-trainable params: 4,052,131 (15.46 MB)

E0000 00:00:1767451890.495607      24 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1767451893.444950      79 cuda_dnn.cc:529] Loaded cuDNN version 91002


1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.0625 - loss: 3.1190


In [17]:
history_1 = model.fit(
    train_ds,
    epochs=EPOCHS_STAGE_1,
    class_weight=class_weight
)


Epoch 1/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 60s 43ms/step - accuracy: 0.2979 - loss: 2.8600
Epoch 2/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.3866 - loss: 1.8495
Epoch 3/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.3997 - loss: 1.7508
Epoch 4/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.3972 - loss: 1.7703
Epoch 5/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.4005 - loss: 1.7242
Epoch 6/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4107 - loss: 1.6926
Epoch 7/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4115 - loss: 1.6666
Epoch 8/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4108 - loss: 1.6698
Epoch 9/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4193 - loss: 1.6717
Epoch 10/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4278 - loss: 1.6186
Epoch 11/15
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4161 - loss: 1.6237
Epoch 12

In [18]:
base_model.trainable = True

# Freeze lower layers to avoid catastrophic forgetting
for layer in base_model.layers[:-40]:
    layer.trainable = False


In [19]:
history_2 = model.fit(
    train_ds,
    epochs=EPOCHS_STAGE_2,
    class_weight=class_weight
)


Epoch 1/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4444 - loss: 1.5736
Epoch 2/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4446 - loss: 1.5660
Epoch 3/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4386 - loss: 1.5416
Epoch 4/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4454 - loss: 1.5665
Epoch 5/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4422 - loss: 1.5405
Epoch 6/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4400 - loss: 1.5592
Epoch 7/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 56s 44ms/step - accuracy: 0.4468 - loss: 1.5212
Epoch 8/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4468 - loss: 1.5067
Epoch 9/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4650 - loss: 1.5052
Epoch 10/10
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 57s 44ms/step - accuracy: 0.4622 - loss: 1.5025


In [20]:
model.save("/kaggle/working/skin_disease.keras")
print("✅ Model saved")


✅ Model saved


In [21]:
import shutil

shutil.make_archive(
    base_name="/kaggle/working/mskin_model",
    format="zip",
    root_dir="/kaggle/working",
    base_dir="skin_disease.keras"  # or folder name
)


'/kaggle/working/mskin_model.zip'